# Load PubMed

In [1]:
import pandas as pd
PubMed = pd.read_excel('../Src/(Cleaned)pubmed-CancerType_Top1-10-set_10000-data.xlsx')
PubMed

,PUMID,Title,Abstract,CancerType,Description,Clean_Description
0,17078348,Understanding the symptoms experienced by indi...,The purpose of this study was to gain a better...,Lung,Understanding the symptoms experienced by indi...,understand symptom experience individual lung ...
1,30206083,Do statins improve outcomes for patients with ...,INTRODUCTION: Lung cancer is the most common n...,Lung,Do statins improve outcomes for patients with ...,statin improve outcome patient non small cell ...
2,22974775,"Lung cancer epidemiology, risk factors, and pr...",The greatest risk by far for developing lung c...,Lung,"Lung cancer epidemiology, risk factors, and pr...",lung cancer epidemiology risk factor preventio...
3,26299737,[Modern Nanomedicine in Treatment of Lung Carc...,BACKGROUNDS: Despite the fast development of n...,Lung,[Modern Nanomedicine in Treatment of Lung Carc...,modern nanomedicine treatment lung carcinoma b...
4,8815254,[Nineteen multiple primary cancer cases of 100...,"In our department, half of 100 consecutive lun...",Lung,[Nineteen multiple primary cancer cases of 100...,nineteen multiple primary cancer case patient ...
...,...,...,...,...,...,...
9995,24122724,High morbidity and mortality found for high-ri...,OBJECTIVES: To give an updated review concerni...,Bladder,High morbidity and mortality found for high-ri...,high morbidity mortality find high risk non mu...
9996,10447660,Case-referent study on occupational risk facto...,OBJECTIVE: To evaluate the possible associatio...,Bladder,Case-referent study on occupational risk facto...,case referent study occupational risk factor b...
9997,3582456,Intravesical irrigation with distilled water d...,"In a retrospective study, the influence of dis...",Bladder,Intravesical irrigation with distilled water d...,intravesical irrigation distill water immediat...
9998,21897260,Ileal neobladder in women with bladder cancer:...,PURPOSE OF REVIEW: Radical cystectomy and urin...,Bladder,Ileal neobladder in women with bladder cancer:...,ileal neobladder woman bladder cancer cancer c...


# Extract TF-IDF Features

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(use_idf=True, ngram_range=(1, 2), min_df=10, max_df=0.8)
tv_matrix = tv.fit_transform(PubMed['Clean_Description'])
tv_matrix

<10000x15814 sparse matrix of type '<class 'numpy.float64'>'
	with 983330 stored elements in Compressed Sparse Row format>

# Cluster Articles Using K-Means

In [3]:
from sklearn.cluster import KMeans

NUM_CLUSTERS = 10
km = KMeans(n_clusters=NUM_CLUSTERS, max_iter=100, n_init=10, random_state=42).fit(tv_matrix)
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [4]:
from collections import Counter

Counter(km.labels_)

Counter({9: 987,
         7: 1051,
         5: 1001,
         6: 996,
         2: 1004,
         8: 1003,
         0: 1013,
         4: 965,
         3: 989,
         1: 991})

In [5]:
PubMed['KMeans_Cluster'] = km.labels_

In [6]:
pubmed_clusters = (PubMed[['PUMID', 'Title', 'KMeans_Cluster']]
                  .sort_values(by=['KMeans_Cluster', 'PUMID'], 
                               ascending=True))
pubmed_clusters

,PUMID,Title,KMeans_Cluster
2895,1296196,"Attributable risk for diet, alcohol, and famil...",0
2676,1320771,[Prevention and surveillance of risk groups fo...,0
2046,1379508,Elective versus emergency surgery for patients...,0
2055,1410067,Colorectal cancer screening.,0
2985,1486445,Intraoperative colonoscopy in patients with co...,0
...,...,...,...
339,32412305,Lung Cancer Mortality and the Availability of ...,9
432,32436293,Sequence of biologic therapies and surgery aff...,9
725,32462835,[Small-cell lung cancer: management and novelt...,9
986,32532368,Gender and lung cancer-SEER-based analysis.,9


In [7]:
pubmed_clusters.to_excel('../Output/KMeans_Cluster_Output.xlsx', index=False) 

In [8]:
pubmed_clusters = (PubMed[['PUMID', 'Title', 'KMeans_Cluster']]
                  .sort_values(by=['KMeans_Cluster', 'PUMID'], 
                               ascending=True)
                  .groupby('KMeans_Cluster').head(20))

In [9]:
feature_names = tv.get_feature_names()
topn_features = 20
ordered_centroids = km.cluster_centers_.argsort()[:, ::-1]

# get key features for each cluster
# get movies belonging to each cluster
for cluster_num in range(NUM_CLUSTERS):
    key_features = [feature_names[index] 
                        for index in ordered_centroids[cluster_num, :topn_features]]
    pubmed_articles = pubmed_clusters[pubmed_clusters['KMeans_Cluster'] == cluster_num]['Title'].values.tolist()
    print('CLUSTER #'+str(cluster_num)+'\n')
    print('Key Features:', key_features)
    print()
    print('PubMed Titles:', pubmed_articles)
    print('-'*80)

CLUSTER #0

Key Features: ['colorectal', 'colorectal cancer', 'crc', 'patient', 'screen', 'stage', 'risk', 'colon', 'survival', 'study', 'tumor', 'expression', 'patient colorectal', 'metastasis', 'year', 'colonoscopy', 'treatment', 'use', 'resection', 'test']

PubMed Titles: ['Attributable risk for diet, alcohol, and family history in the Melbourne Colorectal Cancer Study.', '[Prevention and surveillance of risk groups for colorectal carcinoma].', 'Elective versus emergency surgery for patients with colorectal cancer.', 'Colorectal cancer screening.', 'Intraoperative colonoscopy in patients with colorectal cancer.', '[Multivisceral resection of colorectal cancer--experiences of the Colorectal Cancer Study Group].', 'Predictor of survival for colorectal cancer patients identified.', 'Repeated hepatic resection for recurrent metastases from colorectal cancer.', 'The value of adjuvant therapy after radical surgery for colorectal cancer.', 'Recent advances in the treatment of colorectal ca

In [10]:
from sklearn.metrics import silhouette_score

preds = km.fit_predict(tv_matrix)
score = silhouette_score(tv_matrix, preds)
print('The silhouette score for K-Means with 10 cluster is', score)

The silhouette score for K-Means with 10 cluster is 0.03188815879242259


## Select one article (using "PUMID"), from each cluster and extract top 5 similiarest articles (instances)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tv_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,1.000000,0.061515,0.088812,0.074249,0.097627,0.030607,0.113703,0.062316,0.054583,0.055684,...,0.007435,0.016421,0.029709,0.041221,0.012108,0.012048,0.041185,0.027924,0.011591,0.020964
1,0.061515,1.000000,0.074174,0.088862,0.068625,0.037924,0.171106,0.115116,0.069546,0.135795,...,0.017958,0.044825,0.028957,0.027395,0.018010,0.155452,0.023099,0.028555,0.025803,0.028207
2,0.088812,0.074174,1.000000,0.137289,0.110647,0.058015,0.248487,0.111255,0.080277,0.081217,...,0.009129,0.036299,0.009844,0.015768,0.020715,0.012955,0.225336,0.009698,0.005016,0.007722
3,0.074249,0.088862,0.137289,1.000000,0.109521,0.071664,0.247494,0.093393,0.080238,0.056884,...,0.049775,0.023406,0.009157,0.027696,0.034338,0.012724,0.011421,0.016919,0.010655,0.011032
4,0.097627,0.068625,0.110647,0.109521,1.000000,0.044762,0.157064,0.069573,0.061845,0.044650,...,0.004479,0.018916,0.030967,0.008876,0.036243,0.021370,0.042591,0.033074,0.008619,0.003658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.012048,0.155452,0.012955,0.012724,0.021370,0.012357,0.018038,0.033188,0.015994,0.021941,...,0.054799,0.146896,0.055373,0.103847,0.063855,1.000000,0.067035,0.083979,0.077338,0.112880
9996,0.041185,0.023099,0.225336,0.011421,0.042591,0.007293,0.033225,0.008350,0.024374,0.048804,...,0.030455,0.122245,0.045901,0.038382,0.065740,0.067035,1.000000,0.094757,0.057887,0.060383
9997,0.027924,0.028555,0.009698,0.016919,0.033074,0.014134,0.010902,0.015690,0.012249,0.041560,...,0.096783,0.156038,0.059505,0.044596,0.101673,0.083979,0.094757,1.000000,0.059989,0.067853
9998,0.011591,0.025803,0.005016,0.010655,0.008619,0.010631,0.014759,0.002436,0.007899,0.016626,...,0.030624,0.142499,0.102864,0.058796,0.212628,0.077338,0.057887,0.059989,1.000000,0.039349


In [12]:
pubmed_list = PubMed['Title'].values
pubmed_list, pubmed_list.shape

(array(['Understanding the symptoms experienced by individuals with lung cancer.',
        'Do statins improve outcomes for patients with non-small cell lung cancer? A systematic review and meta-analysis protocol.',
        'Lung cancer epidemiology, risk factors, and prevention.', ...,
        'Intravesical irrigation with distilled water during and immediately after transurethral resection and later for superficial bladder cancer.',
        'Ileal neobladder in women with bladder cancer: cancer control and functional aspects.',
        'Molecular Subtype Profiling of Urothelial Carcinoma Using a Subtype-Specific Immunohistochemistry Panel.'],
       dtype=object), (10000,))

In [13]:
def similar_articles(pubmed_title, article=pubmed_list, doc_sims=doc_sim_df):
    # find pubmed index
    pubmed_idx = np.where(article == pubmed_title)[0][0]
    # get article similarities
    article_similarities = doc_sims.iloc[pubmed_idx].values
    # get top 5 similar pubmed article IDs
    similar_pubmed_idxs = np.argsort(-article_similarities)[1:6]
    # get top 5 article
    similar_article = article[similar_pubmed_idxs]
    # return the top 5 similar articles
    return similar_article

In [14]:
articles= []
articles_cluster = []
for i in range(NUM_CLUSTERS):
    pubmed_articles = pubmed_clusters[pubmed_clusters['KMeans_Cluster'] == i]['Title'].values.tolist()
    articles.append(pubmed_articles[0])
    articles_cluster.append(i)
articles

['Attributable risk for diet, alcohol, and family history in the Melbourne Colorectal Cancer Study.',
 'The local, regional and systemic attack on bladder cancer.',
 '[Cytological diagnosis of stomach cancer].',
 '[Association of tuberculosis and thyroid cancer. Value of exploratory cervicotomy].',
 'Hepatocellular carcinoma in the Rhodesian African.',
 '[First results of the work of the urban cytological laboratory].',
 'Epidemiology of prostate cancer with special reference to the role of diet.',
 '[Esophago-intestinal bypass anastomoses for inoperable cancer of the proximal portion of the stomach and abdominal portion of the esophagus].',
 'Letter: Prevalence and duration of undetected breast cancer.',
 'Letter: Mortality from lung cancer.']

In [15]:
data = {'Title': [], 'KMeans_cluster': [], 'Similar_Title': [], 'Similar_cluster': []}
df_similar = pd.DataFrame(data)
df_similar

,Title,KMeans_cluster,Similar_Title,Similar_cluster


In [17]:
import numpy as np

cluster = 0
for article in articles:
    print('Article Title from Cluster', cluster, ':', article)
    print()
    print('Top 5 similar article:')
    five_similar_articles = similar_articles(pubmed_title=article)
    for one_article in five_similar_articles:
        article_details = PubMed.loc[PubMed['Title'] == one_article]
        print('From Cluster', article_details['KMeans_Cluster'].tolist()[0], 'Title :', one_article)
        new_row = {'Title': articles[cluster], 'KMeans_cluster': articles_cluster[cluster], 'Similar_Title': one_article, 'Similar_cluster': article_details['KMeans_Cluster'].tolist()[0]}
        df_similar = df_similar.append(new_row, ignore_index=True)
    print('-'*80)
    print()
    cluster+=1

Article Title from Cluster 0 : Attributable risk for diet, alcohol, and family history in the Melbourne Colorectal Cancer Study.

Top 5 similar article:
From Cluster 0 Title : Re: improving the cost-effectiveness of colorectal cancer screening.
From Cluster 0 Title : [Significance of family anamnesis in colorectal carcinoma].
From Cluster 0 Title : Colorectal cancer: molecules and populations.
From Cluster 7 Title : Superficial spreading carcinoma of the stomach.
From Cluster 0 Title : Ranitidine as adjuvant treatment in colorectal cancer (Br J Surg 2002; 89:
--------------------------------------------------------------------------------

Article Title from Cluster 1 : The local, regional and systemic attack on bladder cancer.

Top 5 similar article:
From Cluster 1 Title : Current trends in bladder cancer in England and Wales.
From Cluster 1 Title : Bladder cancer: diagnosis and management of bladder cancer: © NICE (2015) Bladder cancer: diagnosis and management of bladder cancer.
Fro

In [18]:
df_similar

,Title,KMeans_cluster,Similar_Title,Similar_cluster
0,"Attributable risk for diet, alcohol, and famil...",0.0,Re: improving the cost-effectiveness of colore...,0.0
1,"Attributable risk for diet, alcohol, and famil...",0.0,[Significance of family anamnesis in colorecta...,0.0
2,"Attributable risk for diet, alcohol, and famil...",0.0,Colorectal cancer: molecules and populations.,0.0
3,"Attributable risk for diet, alcohol, and famil...",0.0,Superficial spreading carcinoma of the stomach.,7.0
4,"Attributable risk for diet, alcohol, and famil...",0.0,Ranitidine as adjuvant treatment in colorectal...,0.0
5,"The local, regional and systemic attack on bla...",1.0,Current trends in bladder cancer in England an...,1.0
6,"The local, regional and systemic attack on bla...",1.0,Bladder cancer: diagnosis and management of bl...,1.0
7,"The local, regional and systemic attack on bla...",1.0,Significance of age and comorbidity as prognos...,1.0
8,"The local, regional and systemic attack on bla...",1.0,[Epidemiology of bladder cancer].,1.0
9,"The local, regional and systemic attack on bla...",1.0,XPC epigenetic silence coupled with p53 altera...,1.0


In [29]:
pubmed_idx = np.where(pubmed_list == '[First results of the work of the urban cytological laboratory].')[0][0]
PubMed.loc[[pubmed_idx]]

,PUMID,Title,Abstract,CancerType,Description,Clean_Description,KMeans_Cluster
7698,1946,[First results of the work of the urban cytolo...,BACKGROUND: The inhibitory effects of N-(4-hyd...,Cervix Uteri,[First results of the work of the urban cytolo...,﻿1 result work urban cytological laboratory ba...,5


In [36]:
print('Title')
print(PubMed.loc[[pubmed_idx]]['Title'].tolist())
print('Clean Description')
print(PubMed.loc[[pubmed_idx]]['Clean_Description'].tolist())

Title
['[First results of the work of the urban cytological laboratory].']
Clean Description
['\ufeff1 result work urban cytological laboratory background inhibitory effect n hydroxyphenyl retinamide hpr tumorigenesis tumor growth may result ability induce apoptosis programme cell death since antioxidant inhibit hpr induce apoptosis experiment plan determine whether level reactive oxygen species increase cell undergo apoptosis exposure hpr method cell human cervical carcinoma cell line c normal human cervical epithelial cell treat hpr analyze survival induction apoptosis generation reactive oxygen species expression apoptosis relate protein bcl bax result treatment hpr decrease c cell numb induce apoptosis time dose dependent fashion dna fragmentation typical apoptosis observe cell expose hpr concentration microm high hour generation reactive oxygen species enhance']
